# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 5 2022 1:27PM,241107,102,6675999,"Senseonics, Incorporated",Released
1,Jul 5 2022 1:22PM,241106,12,LF-FLX600010,LF of America Corp.,Released
2,Jul 5 2022 1:08PM,241103,12,HH-35625,Hush Hush,Released
3,Jul 5 2022 1:08PM,241103,12,HH6338.1,Hush Hush,Released
4,Jul 5 2022 1:08PM,241103,12,HH6339,Hush Hush,Released
5,Jul 5 2022 1:08PM,241103,12,HH-35385,Hush Hush,Released
6,Jul 5 2022 1:08PM,241103,12,HH-35394,Hush Hush,Released
7,Jul 5 2022 1:08PM,241103,12,HH-35417,Hush Hush,Released
8,Jul 5 2022 1:08PM,241103,12,HH-35442,Hush Hush,Released
9,Jul 5 2022 1:08PM,241103,12,HH-35503,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
67,241103,Released,15
68,241104,Released,1
69,241105,Executing,2
70,241106,Released,1
71,241107,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241103,NaN,NaN,15.0
241104,NaN,NaN,1.0
241105,NaN,2.0,NaN
241106,NaN,NaN,1.0
241107,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241010,0.0,0.0,60.0
241016,0.0,0.0,60.0
241017,0.0,0.0,60.0
241019,0.0,0.0,55.0
241022,59.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241010,0,0,60
241016,0,0,60
241017,0,0,60
241019,0,0,55
241022,59,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241010,0,0,60
1,241016,0,0,60
2,241017,0,0,60
3,241019,0,0,55
4,241022,59,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241010,,,60
1,241016,,,60
2,241017,,,60
3,241019,,,55
4,241022,59,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 5 2022 1:27PM,241107,102,"Senseonics, Incorporated"
1,Jul 5 2022 1:22PM,241106,12,LF of America Corp.
2,Jul 5 2022 1:08PM,241103,12,Hush Hush
17,Jul 5 2022 1:06PM,241105,19,"AdvaGen Pharma, Ltd"
19,Jul 5 2022 1:06PM,241104,12,Hush Hush
20,Jul 5 2022 1:03PM,241100,16,"SHL Pharma, LLC"
21,Jul 5 2022 1:01PM,241102,10,Emerginnova
23,Jul 5 2022 1:01PM,241099,10,Beach Patient Assistance
26,Jul 5 2022 1:01PM,241099,10,Beach Products Inc
29,Jul 5 2022 1:00PM,241096,12,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 5 2022 1:27PM,241107,102,"Senseonics, Incorporated",,,1
1,Jul 5 2022 1:22PM,241106,12,LF of America Corp.,,,1
2,Jul 5 2022 1:08PM,241103,12,Hush Hush,,,15
3,Jul 5 2022 1:06PM,241105,19,"AdvaGen Pharma, Ltd",,2,
4,Jul 5 2022 1:06PM,241104,12,Hush Hush,,,1
5,Jul 5 2022 1:03PM,241100,16,"SHL Pharma, LLC",,,1
6,Jul 5 2022 1:01PM,241102,10,Emerginnova,,,2
7,Jul 5 2022 1:01PM,241099,10,Beach Patient Assistance,,,6
8,Jul 5 2022 1:01PM,241099,10,Beach Products Inc,,,6
9,Jul 5 2022 1:00PM,241096,12,"NBTY Global, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 5 2022 1:27PM,241107,102,"Senseonics, Incorporated",1,,
1,Jul 5 2022 1:22PM,241106,12,LF of America Corp.,1,,
2,Jul 5 2022 1:08PM,241103,12,Hush Hush,15,,
3,Jul 5 2022 1:06PM,241105,19,"AdvaGen Pharma, Ltd",,2,
4,Jul 5 2022 1:06PM,241104,12,Hush Hush,1,,
5,Jul 5 2022 1:03PM,241100,16,"SHL Pharma, LLC",1,,
6,Jul 5 2022 1:01PM,241102,10,Emerginnova,2,,
7,Jul 5 2022 1:01PM,241099,10,Beach Patient Assistance,6,,
8,Jul 5 2022 1:01PM,241099,10,Beach Products Inc,6,,
9,Jul 5 2022 1:00PM,241096,12,"NBTY Global, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 5 2022 1:27PM,241107,102,"Senseonics, Incorporated",1,,
1,Jul 5 2022 1:22PM,241106,12,LF of America Corp.,1,,
2,Jul 5 2022 1:08PM,241103,12,Hush Hush,15,,
3,Jul 5 2022 1:06PM,241105,19,"AdvaGen Pharma, Ltd",,2,
4,Jul 5 2022 1:06PM,241104,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 5 2022 1:27PM,241107,102,"Senseonics, Incorporated",1.0,NaN,NaN
1,Jul 5 2022 1:22PM,241106,12,LF of America Corp.,1.0,NaN,NaN
2,Jul 5 2022 1:08PM,241103,12,Hush Hush,15.0,NaN,NaN
3,Jul 5 2022 1:06PM,241105,19,"AdvaGen Pharma, Ltd",NaN,2.0,NaN
4,Jul 5 2022 1:06PM,241104,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 5 2022 1:27PM,241107,102,"Senseonics, Incorporated",1.0,0.0,0.0
1,Jul 5 2022 1:22PM,241106,12,LF of America Corp.,1.0,0.0,0.0
2,Jul 5 2022 1:08PM,241103,12,Hush Hush,15.0,0.0,0.0
3,Jul 5 2022 1:06PM,241105,19,"AdvaGen Pharma, Ltd",0.0,2.0,0.0
4,Jul 5 2022 1:06PM,241104,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,11329526,1385.0,57.0,60.0
102,241107,1.0,0.0,0.0
12,2169787,52.0,41.0,0.0
15,241073,19.0,0.0,0.0
16,241100,1.0,0.0,0.0
19,482162,1.0,2.0,0.0
20,723232,62.0,20.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,11329526,1385.0,57.0,60.0
1,102,241107,1.0,0.0,0.0
2,12,2169787,52.0,41.0,0.0
3,15,241073,19.0,0.0,0.0
4,16,241100,1.0,0.0,0.0
5,19,482162,1.0,2.0,0.0
6,20,723232,62.0,20.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,1385.0,57.0,60.0
1,102,1.0,0.0,0.0
2,12,52.0,41.0,0.0
3,15,19.0,0.0,0.0
4,16,1.0,0.0,0.0
5,19,1.0,2.0,0.0
6,20,62.0,20.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1385.0
1,102,Released,1.0
2,12,Released,52.0
3,15,Released,19.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20
Status,,,,,,,
Completed,60.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,57.0,0.0,41.0,0.0,0.0,2.0,20.0
Released,1385.0,1.0,52.0,19.0,1.0,1.0,62.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20
0,Completed,60.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,57.0,0.0,41.0,0.0,0.0,2.0,20.0
2,Released,1385.0,1.0,52.0,19.0,1.0,1.0,62.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20
0,Completed,60.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,57.0,0.0,41.0,0.0,0.0,2.0,20.0
2,Released,1385.0,1.0,52.0,19.0,1.0,1.0,62.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()